Firstly, we will generate answers to the questions using [OFA](https://github.com/OFA-Sys/OFA/tree/main) model

In [ ]:
!git clone https://github.com/OFA-Sys/OFA.git

Download Checkpoint

In [ ]:
!mkdir -p OFA/checkpoints/
!wget https://ofa-silicon.oss-us-west-1.aliyuncs.com/checkpoints/ofa_large_384.pt
!mv ofa_large_384.pt OFA/checkpoints/ofa_large_384.pt

Install Fairseq

In [ ]:
!git clone https://github.com/pytorch/fairseq.git -b v0.12.0

In [ ]:
cd fairseq

In [ ]:
!pip -q install --use-feature=no-binary-enable-wheel-cache ./

In [ ]:
cd ../OFA

In [ ]:
%%capture
!sed '1d' requirements.txt | xargs -I {} pip install {}

In [ ]:
!pip -q install wget

Import required packages, specify some generation options and check whether to use GPU or FP16. 

In [ ]:
import torch
import numpy as np
import re
from fairseq import checkpoint_utils
from fairseq import distributed_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from utils.zero_shot_utils import zero_shot_step
from tasks.mm_tasks.vqa_gen import VqaGenTask
from PIL import Image
import pandas as pd
import os
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import wget

In [ ]:
# Register VQA task
tasks.register_task('vqa_gen',VqaGenTask)

# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False

# specify some options for evaluation
parser = options.get_generation_parser()
input_args = ["", "--task=vqa_gen", "--beam=100", "--unnormalized", "--path=checkpoints/ofa_large_384.pt", "--bpe-dir=utils/BPE"]
args = options.parse_args_and_arch(parser, input_args)
cfg = convert_namespace_to_omegaconf(args)

### **Build Model**
Let's build a model, load the weights from the checkpoint, and also build a generator.

In [ ]:
# Load pretrained ckpt & config
task = tasks.setup_task(cfg.task)
models, cfg = checkpoint_utils.load_model_ensemble(
    utils.split_paths(cfg.common_eval.path),
    task=task
)

# Move models to GPU
for model in models:
    model.eval()
    if use_fp16:
        model.half()
    if use_cuda and not cfg.distributed_training.pipeline_model_parallel:
        model.cuda()
    model.prepare_for_inference_(cfg)

# Initialize generator
generator = task.build_generator(models, cfg.generation)

### **Preprocess**
Define the required transformation fucntions for preprocessing inputs.

In [ ]:
# Image transform
from torchvision import transforms
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

patch_resize_transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((cfg.task.patch_image_size, cfg.task.patch_image_size), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# Text preprocess
bos_item = torch.LongTensor([task.src_dict.bos()])
eos_item = torch.LongTensor([task.src_dict.eos()])
pad_idx = task.src_dict.pad()

# Normalize the question
def pre_question(question, max_ques_words):
    question = question.lower().lstrip(",.!?*#:;~").replace('-', ' ').replace('/', ' ')
    question = re.sub(
        r"\s{2,}",
        ' ',
        question,
    )
    question = question.rstrip('\n')
    question = question.strip(' ')
    # truncate question
    question_words = question.split(' ')
    if len(question_words) > max_ques_words:
        question = ' '.join(question_words[:max_ques_words])
    return question

def encode_text(text, length=None, append_bos=False, append_eos=False):
    s = task.tgt_dict.encode_line(
        line=task.bpe.encode(text),
        add_if_not_exist=False,
        append_eos=False
    ).long()
    if length is not None:
        s = s[:length]
    if append_bos:
        s = torch.cat([bos_item, s])
    if append_eos:
        s = torch.cat([s, eos_item])
    return s

# Construct input for open-domain VQA task
def construct_sample(image: Image, question: str):
    patch_image = patch_resize_transform(image).unsqueeze(0)
    patch_mask = torch.tensor([True])

    question = pre_question(question, task.cfg.max_src_length)
    question = question + '?' if not question.endswith('?') else question
    src_text = encode_text(' {}'.format(question), append_bos=True, append_eos=True).unsqueeze(0)

    src_length = torch.LongTensor([s.ne(pad_idx).long().sum() for s in src_text])
    ref_dict = np.array([{'yes': 1.0}]) # just placeholder
    sample = {
        "id":np.array(['42']),
        "net_input": {
            "src_tokens": src_text,
            "src_lengths": src_length,
            "patch_images": patch_image,
            "patch_masks": patch_mask,
        },
        "ref_dict": ref_dict,
    }
    return sample
  
# Function to turn FP32 to FP16
def apply_half(t):
    if t.dtype is torch.float32:
        return t.to(dtype=torch.half)
    return t

### **Run Inference**
Download images and run the following scripts to generate answers to the questions

In [ ]:
test_private = pd.read_csv(
    'https://raw.githubusercontent.com/Toloka/WSDMCup2023/main/test_private.csv')

In [ ]:
os.mkdir('../imgs/')
img_paths = Parallel(
    n_jobs=100)(delayed(wget.download)(img_url, out='../imgs') for img_url in tqdm(test_private.image)
)

In [ ]:
answers = []
for i, row in tqdm(enumerate(test_private.iterrows()), total=len(test_private)):
    row = row[1]
    image_path = row['image'].split('/')[-1]
    image = Image.open(os.path.join('../imgs', image_path))
    question = row['question'] + ' Name an object in the picture'

    # Construct input sample & preprocess for GPU if cuda available
    sample = construct_sample(image, question)
    sample = utils.move_to_cuda(sample) if use_cuda else sample
    sample = utils.apply_to_sample(apply_half, sample) if use_fp16 else sample

    # Run eval step for open-domain VQA
    with torch.no_grad():
        try:
            result, scores = zero_shot_step(task, generator, models, sample)
            answer = result[0]['answer']
        except ValueError:
            print('ValueError')
            answer = 'ValueError'
    answers.append(answer)

In [ ]:
test_private['answer'] = answers
test_private = test_private[test_private['answer'] != 'ValueError']
test_private.to_csv('../vqa_answers.csv', index=False)

That's it! Now download `vqa_answers.csv` and run `image_grounding_sam.ipynb` notebook to predict bounding boxes